In [1]:
import numpy as np
import operator
from collections import defaultdict

In [2]:
def get_lower_bound(noisy):
    def important_pair_of_columns_in_conflict(D):
        important_columns = defaultdict(lambda: 0)
        for p in range(D.shape[1]):
            for q in range(p + 1, D.shape[1]):
                oneone = 0
                zeroone = 0
                onezero = 0
                for r in range(D.shape[0]):
                    if D[r,p] == 1 and D[r,q] == 1:
                        oneone += 1
                    if D[r,p] == 0 and D[r,q] == 1:
                        zeroone += 1
                    if D[r,p] == 1 and D[r,q] == 0:
                        onezero += 1
                if oneone*zeroone*onezero > 0:
                    important_columns[(p,q)] += oneone*zeroone*onezero
        return important_columns
    
    def get_partinion(D):
        ipofic = important_pair_of_columns_in_conflict(D)
        if len(ipofic) == 0:
            return []
        sorted_ipofic = sorted(ipofic.items(), key=operator.itemgetter(1), reverse=True)
        pairs = [sorted_ipofic[0][0]]
        elements = [sorted_ipofic[0][0][0], sorted_ipofic[0][0][1]]
        sorted_ipofic.remove(sorted_ipofic[0])
        for x in sorted_ipofic[:]:
            notFound = True
            for y in x[0]:
                if y in elements:
                    sorted_ipofic.remove(x)
                    notFound = False
                    break
            if notFound:
                pairs.append(x[0])
                elements.append(x[0][0])
                elements.append(x[0][1])
        #print(sorted_ipofic, pairs, elements)
        partitions = []
        for x in pairs:
            partitions.append(D[:,x])
        return partitions
    
    def lower_bound_for_a_pair_of_columns(D):
        foundOneOne = False
        numberOfZeroOne = 0
        numberOfOneZero = 1
        for r in range(D.shape[0]):
            if D[r,0] == 1 and D[r,1] == 1:
                foundOneOne = True
            if D[r,0] == 0 and D[r,1] == 1:
                numberOfZeroOne += 1
            if D[r,0] == 1 and D[r,1] == 0:
                numberOfOneZero += 1
        if foundOneOne:
            if numberOfZeroOne*numberOfOneZero > 0:
                return min(numberOfZeroOne, numberOfOneZero)
        return None
    
    LB = []
    for D in get_partinion(noisy):
        LB.append(lower_bound_for_a_pair_of_columns(D))
    return sum(LB)

In [3]:
noisy = np.array([
    [0,1,1,0],
    [1,0,0,1],
    [1,1,0,0],
    [0,0,1,0]
])

get_lower_bound(noisy)

1